In [55]:
import pandas as pd
import numpy as np
import simplekml


In [ ]:
# Open NAV_BASE file
nav_base = pd.read_csv('FAA_data_20250123/NAV_BASE.csv')
#Filter dataframe by NAV_TYPE
vor = nav_base[nav_base['NAV_TYPE'].isin(['VOR', 'TACAN', 'VOR/DME', 'VORTAC'])]

#Create Low Alt KML
low_kml = simplekml.Kml()
low_kml.document.name = 'Low Altitude VOR Stations'

#Create High Alt KML
high_kml = simplekml.Kml()
high_kml.document.name = 'High Altitude VOR Stations'

In [67]:
alt_dict = {'low':['L','VL'], 'high':['H','VH']}

In [58]:
def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

In [93]:
def write_kml(kml,height):
    blue_style = simplekml.Style()
    blue_style.iconstyle.color = simplekml.Color.blue
    blue_style.iconstyle.scale = 1.0
    blue_style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/blu-circle.png'
    # Add the style to the KML document
    
    #Add yellow style
    yellow_style = simplekml.Style()
    yellow_style.iconstyle.color = simplekml.Color.yellow
    yellow_style.iconstyle.scale = 1.0
    yellow_style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/ylw-circle.png'
    
    df = vor[vor['ALT_CODE'].isin(alt_dict[height])]
    for index, row in df.iterrows():
        lat = dms_to_decimal(row['LAT_DEG'], row['LAT_MIN'], row['LAT_SEC'], row['LAT_HEMIS'])
        lon = dms_to_decimal(row['LONG_DEG'], row['LONG_MIN'], row['LONG_SEC'], row['LONG_HEMIS'])
        mag_deviation = f"{row['MAG_VARN_HEMIS']}{int(row['MAG_VARN'])}°"
        description = (
            f"Altitude Code: {row['ALT_CODE']}<br>"
            f"Magnetic Deviation: {mag_deviation}<br>"
            f"City: {row['CITY']}, {row['STATE_CODE']}<br>"
            f"Frequency: {row['FREQ']} MHz"
        )
        pnt = kml.newpoint(name=row['NAV_ID'],
                        coords=[(lon, lat)],)
        pnt.description = description
        if height == 'low':
            pnt.style = yellow_style
        else:
            pnt.style = blue_style

In [96]:
write_kml(high_kml,'high')
high_kml.save('high_altitude_vor.kml')
write_kml(low_kml,'low')
low_kml.save('low_altitude_vor.kml')